In [2]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import os
import json
import numpy as np

import pytz
from datetime import datetime, timedelta


alt.renderers.enable('notebook')
%matplotlib inline
pst = pytz.timezone('US/Pacific')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)


In [3]:

import sklearn.ensemble as sk_ens
import sklearn.neighbors as sk_neigh
import sklearn.metrics as sk_met
import sklearn.model_selection as sk_model

In [4]:
train = pd.read_csv(os.path.expanduser('~/dev/datasets/mnist/mnist_train.csv'), header=None)
test = pd.read_csv(os.path.expanduser('~/dev/datasets/mnist/mnist_test.csv'), header=None)
train.columns = map(str, train.columns)
test.columns = map(str, test.columns)

In [5]:
X_train = train[[col for col in train.columns if col not in ['0']]]
y_train = train['0']
X_test = test[[col for col in test.columns if col not in ['0']]]
y_test = test['0']

In [6]:
X_test.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84,185,159,151,60,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,222,254,254,254,254,241,198,198,198,198,198,198,198,198,170,52,0,0,0,0,0,0,0,0,0,0,0,0,67,114,72,114,163,227,254,225,254,254,254,250,229,254,254,140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,66,14,67,67,67,59,21,236,254,106,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,83,253,209,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,233,255,83,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,129,254,238,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,59,249,254,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [9]:
adaboost = sk_ens.AdaBoostClassifier(n_estimators=100, random_state=0)
knn = sk_neigh.KNeighborsClassifier(n_jobs=10)
rand_for = sk_ens.RandomForestClassifier(n_jobs=10, random_state=0, n_estimators=200, min_samples_leaf=1)

In [10]:
models = [rand_for, adaboost, knn]

In [11]:
def scorer(model, X, y):
    y_pred = model.predict(X)
    return sk_met.f1_score(y, y_pred, average='weighted')

In [12]:
print(datetime.now(tz=pst))
class_reports = []
for model in models:
    print('Starting: ', datetime.now(tz=pst))
    print(model)
    model.fit(X_train, y_train)
    cv_score = sk_model.cross_val_score(model, X_train, y_train, cv=5, n_jobs=10, scoring=scorer)
    print('CV Score: Mean {0:.4f}, {1}'.format(np.mean(cv_score), np.round(cv_score, 4)))

    test_preds = model.predict(X_test)
    cr = sk_met.classification_report(y_test, test_preds, output_dict=True)
    cr = pd.DataFrame(cr)
    display(cr)
    class_reports.append(cr)
    print('Ending: ', datetime.now(tz=pst))
    

2018-10-03 17:19:42.170546-07:00
Starting:  2018-10-03 17:19:42.170809-07:00
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=10,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
CV Score: Mean 0.9676, [0.9706 0.967  0.9657 0.9638 0.9711]


,0,1,2,3,4,5,6,7,8,9,micro avg,macro avg,weighted avg
f1-score,0.980273,0.989863,0.966184,0.966436,0.975013,0.969014,0.976067,0.967237,0.961320,0.958333,0.9712,0.970974,0.971186
precision,0.971916,0.990300,0.963391,0.963583,0.976507,0.973952,0.973029,0.972468,0.965803,0.959285,0.9712,0.971023,0.971207
recall,0.988776,0.989427,0.968992,0.969307,0.973523,0.964126,0.979123,0.962062,0.956879,0.957384,0.9712,0.970960,0.971200
support,980.000000,1135.000000,1032.000000,1010.000000,982.000000,892.000000,958.000000,1028.000000,974.000000,1009.000000,10000.0000,10000.000000,10000.000000


Ending:  2018-10-03 17:20:48.424056-07:00
Starting:  2018-10-03 17:20:48.424169-07:00
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=0)
CV Score: Mean 0.7175, [0.7193 0.7348 0.7179 0.6906 0.7247]


,0,1,2,3,4,5,6,7,8,9,micro avg,macro avg,weighted avg
f1-score,0.857004,0.892576,0.648649,0.653016,0.767096,0.642487,0.745887,0.721588,0.674498,0.624558,0.7296,0.722736,0.725177
precision,0.816251,0.843260,0.797737,0.740276,0.747104,0.660355,0.663415,0.736575,0.644528,0.637113,0.7296,0.728661,0.731403
recall,0.902041,0.948018,0.546512,0.584158,0.788187,0.625561,0.851775,0.707198,0.707392,0.612488,0.7296,0.727333,0.729600
support,980.000000,1135.000000,1032.000000,1010.000000,982.000000,892.000000,958.000000,1028.000000,974.000000,1009.000000,10000.0000,10000.000000,10000.000000


Ending:  2018-10-03 17:23:13.571122-07:00
Starting:  2018-10-03 17:23:13.571194-07:00
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=10, n_neighbors=5, p=2,
           weights='uniform')


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/jeevs/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 899, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "/Users/jeevs/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/_parallel_backends.py", line 517, in wrap_future_result
    return future.result(timeout=timeout)
  File "/Users/jeevs/anaconda3/lib/python3.6/concurrent/futures/_base.py", line 427, in result
    self._condition.wait(timeout)
  File "/Users/jeevs/anaconda3/lib/python3.6/threading.py", line 295, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jeevs/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-02c3d7a85c77>", line 7, in <module>
    cv_score = sk_model.

TypeError: must be str, not list